In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('..')

In [ ]:
from audio_file import read_samples, write_samples, read_concat_samples
from signal_processing.features import extract_features
from signal_processing.windowing import window_samples
from grain_cloud import GrainCloud
from audio_clip import AudioClip, clips_from_folder
from utils import get_files_of_types
import definitions
from manage_packs import update_samplepack_db
from signal_processing.processors import most_dissimilar_segments, strip_silence, cross_fade_all
from signal_processing import processors

from copy import deepcopy

In [ ]:
# directory = "E:/tsvr-samplepack-tools/data/raw/Environments/CitySounds"
directory = "E:/AudioClips/MyMusic/Gnuit/vox"
files = get_files_of_types(directory, definitions.AUDIO_FILE_TYPES)
clips = clips_from_folder(directory, random_subset=10)

In [ ]:
clips_cloud = deepcopy(clips)
ws = 16384
clip = clips_cloud[3]
clip.apply_processor(lambda x: most_dissimilar_segments(x,
                                                        n_mfcc=5,
                                                        window_size=ws,
                                                        hop_size=ws,
                                                        top_ratio=0.95,
                                                        crossfade_ratio=0.1))
clip.display()

In [ ]:
clip = AudioClip.from_file("E:/AudioClips/the factory.wav")
ws = 8192
clip.apply_processor(lambda x: most_dissimilar_segments(x,
                                                        n_mfcc=28,
                                                        window_size=ws,
                                                        hop_size=ws,
                                                        top_ratio=0.05,
                                                        crossfade_ratio=0.05))
clip.display()

In [ ]:
from signal_processing.processors import strip_silence, cross_fade_all
from copy import deepcopy

clips_cloud = deepcopy(clips)
cloud = GrainCloud(clips_cloud)
ws = 16384

new_clip = cloud.render(
    clip_processors=[lambda x: strip_silence(x, -40, 0), lambda x: most_dissimilar_segments(x,
                                                        n_mfcc=5,
                                                        window_size=16384,
                                                        hop_size=16384,
                                                        top_ratio=0.95,
                                                        crossfade_ratio=0.1)],
    consolidate_processor=lambda x: cross_fade_all(x, 10000)
)

new_clip.display()

Export the GrainCloud to the samplepacks root, which also includes metadata

In [ ]:
cloud.export_as_samplepack("City",
            clip_processors=[lambda x: strip_silence(x, -30, 0)],
            consolidate_processor=lambda x: cross_fade_all(x, 10000))

In [ ]:
update_samplepack_db()

Idea - sort samples by first component of MFCC to improve the granulation, timbre separation

In [ ]:
ws = 16384
clip = AudioClip.from_file("E:/AudioClips/the factory.wav")
# clip = AudioClip.from_file("E:/AudioClips/MyMusic/jazz_w_kali.wav")
clip.apply_processor(lambda x: processors.strip_silence(x, -50, 1000))
clip.apply_processor(lambda x: processors.most_dissimilar_segments(x,
                                                        n_mfcc=10,
                                                        window_size=ws,
                                                        hop_size=ws,
                                                        top_ratio=0.15,
                                                        crossfade_ratio=0.1))
clip.apply_processor(lambda x: processors.normalize(x))
clip.display()  

In [ ]:
directory = "E:/AudioClips/MyMusic/Gnuit/wav"
files = get_files_of_types(directory, definitions.AUDIO_FILE_TYPES)
clips = clips_from_folder(directory, random_subset=5)

In [ ]:
# ws = 16384 * 4
ws = 8192*2
cloud = GrainCloud(deepcopy([clip]))
new_clip = cloud.render(
    # clip_processors=[
    #     lambda x: processors.strip_silence(x, -30, 0), 
    #     # lambda x: processors.most_dissimilar_segments(x, 20, ws, ws, 0.5, crossfade_ratio=0.1), 
    #     processors.normalize
    #     ],
    master_processors=[
        lambda x: processors.most_dissimilar_segments(x, 20, ws, ws, 0.5, crossfade_ratio=0.1), 
        processors.normalize
        ],
    consolidate_processor=lambda x: processors.cross_fade_all(x, 1000)
)

new_clip.display()

In [ ]:
df_features = cloud.audio_features()

In [ ]:
df_features

In [ ]:
from analysis import kmeans_quantize_distances

df_features = cloud.audio_features()
kmeans_quantize_distances(df_features, 'mfcc', cluster_ratio=0.3)

In [ ]:
cloud = GrainCloud(cloud.clips[:3])

In [ ]:
fig = cloud.plot_features_3d()

In [ ]:
fig.show()

Idea - make a series of tones that generate MFCC coefficients that look like a given 3D model

In [ ]:
import pandas as pd
from signal_processing.features import windowed_mfccs
import numpy as np
import plotly.express as px

In [ ]:
all_features = []
ws = 16384
hop = 16384
for clip in cloud.clips:
    features = windowed_mfccs(clip.samples[0], window_size=ws, hop_size=hop, n_mfcc=10)
    for win_num, m in enumerate(features.T):
        mfccs = { f'mfcc_{i}': x for i, x in enumerate(m) }
        start_sample = win_num * hop
        end_sample = start_sample + ws
        all_features.append(
            {
                'title' : clip.info['title'],
                'duration' : clip.samples.shape[1],
                'start_sample' : start_sample,
                'end_sample' : end_sample,
                'frac_idx' : (start_sample + (ws // 2)) / clip.samples.shape[1],
                **mfccs
            }
        )

df_features = pd.DataFrame(all_features)
df_features.head()

In [ ]:
px.scatter_3d(df_features, x='mfcc_0', y='mfcc_1', z='mfcc_2', opacity=0.8, size='frac_idx', color='title')